In [1]:
import pandas as pd 
import uproot
import vector

In [4]:
fdir = "/home/llr/cms/kalipoliti/C10630p1_miniAOD/src/HeavyIonsAnalysis/Configuration/test/rootf/"
label = "HiForestMiniAOD_HighPU_1000events"
fname = label + ".root"
rootf = fdir + fname

gen_particles = pd.DataFrame()
gen_particles_cols = ["pt", "eta", "phi", "pdg", "sta"]

tracks = pd.DataFrame()
tracks_cols = ["trkJetId", "trkPt", "trkEta", "trkPhi"]

with uproot.open(rootf) as fin:
    for col in gen_particles_cols:
        branch = fin[f"bDecayAna/hi/{col}"].array(library="pd")
        gen_particles[col] = branch
        
    for col in tracks_cols:
        branch = fin[f"ak4PFJetAnalyzer/t/{col}"].array(library="pd")
        tracks[col] = branch

In [5]:
## Add cuts to remove some of the useless particles
# min_pt = 0.8
# gen_particles = gen_particles[gen_particles["pt"] > min_pt]
gen_particles = gen_particles.reset_index()
gen_particles = gen_particles.rename(columns={"subentry": "id"})
gen_particles

,entry,id,pt,eta,phi,pdg,sta
0,0,0,36.437500,-1.206889,-0.038966,-13,1
1,0,1,38.031250,1.054173,1.478366,13,1
2,0,2,1.270508,-3.984117,-0.045404,2212,1
3,0,3,1.845703,5.114385,-1.167316,2212,1
4,0,4,1.776367,3.800359,-3.121781,2212,1
...,...,...,...,...,...,...,...
221529,999,235,0.380127,-3.383582,2.170863,-211,1
221530,999,236,1.745117,4.466107,3.012587,321,1
221531,999,237,0.231323,5.270660,-2.900379,-211,1
221532,999,238,0.312500,5.298326,0.234672,211,1


In [6]:
tracks = tracks.reset_index()
tracks = tracks.rename(columns={"subentry": "id"})
tracks

,entry,id,trkJetId,trkPt,trkEta,trkPhi
0,0,0,0,4.003906,0.440199,2.019888
1,0,1,0,1.183594,0.422620,1.949563
2,0,2,0,10.140625,0.401196,1.896932
3,0,3,0,6.910156,0.329234,2.006798
4,0,4,0,1.548828,0.054384,1.980132
...,...,...,...,...,...,...
30053,999,36,15,2.636719,-0.228339,1.713240
30054,999,37,15,2.341797,-0.274300,1.440177
30055,999,38,15,1.155273,-0.367138,1.855224
30056,999,39,19,1.144531,-1.275185,2.114997


In [7]:
matched_ids_noCuts = []
matched_ids_medCuts = []

matched_pts_noCuts = []
matched_pts_medCuts = []

matched_etas_noCuts = []
matched_etas_medCuts = []

matched_phis_noCuts = []
matched_phis_medCuts = []

matched_stas_noCuts = []
matched_stas_medCuts = []

for track_row in tracks.itertuples():
    entry_gen_particles = gen_particles[gen_particles["entry"] == track_row.entry]
    min_dr2_noCuts = 1.
    min_dr2_medCuts = 1.
    
    matched_id_noCuts = -1
    matched_id_medCuts = -1

    matched_pt_noCuts = -1.
    matched_pt_medCuts = -1.

    matched_eta_noCuts = -100.
    matched_eta_medCuts = -100.

    matched_phi_noCuts = -100.
    matched_phi_medCuts = -100.

    matched_sta_noCuts = -1
    matched_sta_medCuts = -1
    
    for gen_row in entry_gen_particles.itertuples():
        track_vec = vector.obj(pt=track_row.trkPt, phi=track_row.trkPhi, eta=track_row.trkEta)
        gen_vec = vector.obj(pt=gen_row.pt, phi=gen_row.phi, eta=gen_row.eta)
        
        dr = track_vec.deltaR(gen_vec)
        dr2 = dr * dr
        if dr2 < min_dr2_noCuts:
            min_dr2_noCuts = dr2
            matched_id_noCuts = gen_row.id
            matched_pt_noCuts = gen_row.pt
            matched_eta_noCuts = gen_row.eta
            matched_phi_noCuts = gen_row.phi
            matched_sta_noCuts = gen_row.sta

        rel_pt = track_row.trkPt / gen_row.pt
        if (dr2 < 0.0004) and  (rel_pt > 0.8) and (rel_pt < 1.2):
            if (dr2 < min_dr2_medCuts):
                min_dr2_medCuts = dr2
                matched_id_medCuts = gen_row.id
                matched_pt_medCuts = gen_row.pt
                matched_eta_medCuts = gen_row.eta
                matched_phi_medCuts = gen_row.phi
                matched_sta_medCuts = gen_row.sta

    matched_ids_noCuts.append(matched_id_noCuts)
    matched_ids_medCuts.append(matched_id_medCuts)

    matched_pts_noCuts.append(matched_pt_noCuts)
    matched_pts_medCuts.append(matched_pt_medCuts)

    matched_etas_noCuts.append(matched_eta_noCuts)
    matched_etas_medCuts.append(matched_eta_medCuts)

    matched_phis_noCuts.append(matched_phi_noCuts)
    matched_phis_medCuts.append(matched_phi_medCuts)

    matched_stas_noCuts.append(matched_sta_noCuts)
    matched_stas_medCuts.append(matched_sta_medCuts)
    
tracks["matchIdNoCuts"] = matched_ids_noCuts
tracks["matchIdMedCuts"] = matched_ids_medCuts

tracks["matchPtNoCuts"] = matched_pts_noCuts
tracks["matchPtMedCuts"] = matched_pts_medCuts

tracks["matchEtaNoCuts"] = matched_etas_noCuts
tracks["matchEtaMedCuts"] = matched_etas_medCuts

tracks["matchPhiNoCuts"] = matched_phis_noCuts
tracks["matchPhiMedCuts"] = matched_phis_medCuts

tracks["matchStaNoCuts"] = matched_stas_noCuts
tracks["matchStaMedCuts"] = matched_stas_medCuts

tracks

,entry,id,trkJetId,trkPt,trkEta,trkPhi,matchIdNoCuts,matchIdMedCuts,matchPtNoCuts,matchPtMedCuts,matchEtaNoCuts,matchEtaMedCuts,matchPhiNoCuts,matchPhiMedCuts,matchStaNoCuts,matchStaMedCuts
0,0,0,0,4.003906,0.440199,2.019888,13,13,3.978516,3.978516,0.439720,0.439720,2.019102,2.019102,200,200
1,0,1,0,1.183594,0.422620,1.949563,15,15,1.180664,1.180664,0.422449,0.422449,1.950442,1.950442,200,200
2,0,2,0,10.140625,0.401196,1.896932,11,11,10.343750,10.343750,0.401414,0.401414,1.896151,1.896151,200,200
3,0,3,0,6.910156,0.329234,2.006798,16,16,6.933594,6.933594,0.329298,0.329298,2.006603,2.006603,200,200
4,0,4,0,1.548828,0.054384,1.980132,106,106,1.538086,1.538086,0.055159,0.055159,1.981207,1.981207,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30053,999,36,15,2.636719,-0.228339,1.713240,112,112,2.585938,2.585938,-0.228852,-0.228852,1.712451,1.712451,1,1
30054,999,37,15,2.341797,-0.274300,1.440177,199,199,2.343750,2.343750,-0.274407,-0.274407,1.440079,1.440079,1,1
30055,999,38,15,1.155273,-0.367138,1.855224,198,198,1.156250,1.156250,-0.366369,-0.366369,1.854833,1.854833,1,1
30056,999,39,19,1.144531,-1.275185,2.114997,233,233,1.145508,1.145508,-1.277636,-1.277636,2.111677,2.111677,1,1


In [8]:
tracks.to_csv("./csvs/" + label + ".csv", index=False)

In [9]:
sel = tracks["matchIdNoCuts"] != tracks["matchIdMedCuts"]
sel &= tracks["matchIdMedCuts"] != -1
tracks[sel]

,entry,id,trkJetId,trkPt,trkEta,trkPhi,matchIdNoCuts,matchIdMedCuts,matchPtNoCuts,matchPtMedCuts,matchEtaNoCuts,matchEtaMedCuts,matchPhiNoCuts,matchPhiMedCuts,matchStaNoCuts,matchStaMedCuts
139,6,13,2,4.453125,1.763359,-2.481031,8,5,3.078125,4.437500,1.763778,1.762727,-2.481323,-2.481812,100,100
140,6,14,2,3.115234,1.763176,-2.482208,5,8,4.437500,3.078125,1.762727,1.763778,-2.481812,-2.481323,100,100
353,13,15,1,5.000000,1.890439,2.912200,14,19,8.992188,5.105469,1.892751,1.889330,2.909170,2.908095,200,200
354,13,16,1,9.085938,1.886593,2.906340,19,14,5.105469,8.992188,1.889330,1.892751,2.908095,2.909170,200,200
1092,38,37,3,3.931641,0.251045,-0.784597,9,14,58.750000,4.074219,0.246842,0.249541,-0.780004,-0.775607,100,100
1358,48,5,0,1.532227,-1.047578,-0.429023,18,17,0.687500,1.542969,-1.047212,-1.048311,-0.429596,-0.428144,1,1
1859,65,12,4,1.269531,-2.778893,-2.716286,8,5,0.977539,1.124023,-2.781393,-2.770009,-2.722749,-2.704876,100,100
2982,100,11,4,1.334961,0.221198,-1.437551,31,30,0.831543,1.345703,0.221015,0.219733,-1.437047,-1.437747,1,1
3320,111,5,0,7.609375,-0.442213,-1.293691,20,19,25.296875,7.894531,-0.441481,-0.441664,-1.291738,-1.291640,200,200
3391,113,14,1,1.664062,0.030213,-0.557628,4,5,4.574219,1.672852,0.026917,0.030030,-0.561054,-0.566906,100,100


In [ ]:
oldtracks = tracks